<a href="https://colab.research.google.com/github/Xu-Ruixin/Multilingual-Phoneme-Recognition/blob/main/Czech(cs)_2Fi_of_speechModel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 30 09:04:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U datasets
!pip install -U transformers[torch]
!pip install torchaudio
!pip install librosa
!pip install -U huggingface_hub
!pip install jiwer
!pip install accelerate -U
!pip install phonemizer

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%%capture
!apt install git-lfs
!sudo apt-get install festival espeak-ng mbrola
!apt search mbrola

In [ ]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("facebook/voxpopuli", "cs", split="train")
common_voice_test = load_dataset("facebook/voxpopuli", "fi", split="test")

In [ ]:
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 18902
})

In [ ]:
common_voice_test

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 478
})

In [ ]:
# common_voice_train = common_voice_train.select(list(range(5000)))
common_voice_train = common_voice_train.select(list(range(3000)))
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 3000
})

In [ ]:
# common_voice_test = common_voice_test.select(list(range(500)))
common_voice_test

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 478
})

In [ ]:
common_voice_train = common_voice_train.remove_columns(["accent", "gender"])
common_voice_test = common_voice_test.remove_columns(["accent", "gender"])

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(common_voice_train.remove_columns(["audio_id", "raw_text"]), num_examples=10)

,language,audio,normalized_text,speaker_id,is_gold_transcript
0,8,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f860858b1727f7dfcbcf1817dd95332e8fad098537c56802f61c4de2454c7def/train_part_0/20180705-0900-PLENARY-cs_20180705-09:24:15_4.wav', 'array': [0.0009765625, 0.002197265625, -0.00128173828125, -0.000274658203125, 0.00201416015625, -0.002410888671875, -0.00250244140625, 0.00189208984375, 0.003143310546875, 9.1552734375e-05, -0.0015869140625, 0.000579833984375, 0.00213623046875, 0.002105712890625, -0.0020751953125, -0.003936767578125, 0.001800537109375, 0.004058837890625, -0.00030517578125, -0.000946044921875, 0.001861572265625, 0.00225830078125, -0.000640869140625, -0.003692626953125, -0.0009765625, 0.002777099609375, 0.00079345703125, -0.00115966796875, 0.00067138671875, 0.001983642578125, -0.0015869140625, -0.002685546875, 0.000762939453125, 0.002838134765625, 0.0008544921875, -0.00311279296875, -0.00091552734375, 0.0015869140625, -0.0010986328125, -0.001861572265625, -0.000152587890625, 0.001800537109375, 0.001251220703125, -0.004669189453125, -0.00439453125, 0.001983642578125, 0.00299072265625, -0.000213623046875, -0.003021240234375, -0.0028076171875, 0.00079345703125, 0.002838134765625, 0.000885009765625, -0.00152587890625, -0.00103759765625, 0.000335693359375, 0.000762939453125, 0.000335693359375, 0.000640869140625, 0.00018310546875, -0.000579833984375, 0.002960205078125, 0.00030517578125, -0.003570556640625, 0.002044677734375, 0.004241943359375, 0.0001220703125, -0.00140380859375, -0.00152587890625, 0.00103759765625, 0.00341796875, -0.000518798828125, -0.00177001953125, 0.00286865234375, 0.00347900390625, -0.00164794921875, -0.00286865234375, 0.001220703125, 0.0013427734375, -0.0010986328125, 3.0517578125e-05, 0.000701904296875, 0.000732421875, -0.0020751953125, -0.001739501953125, 0.00067138671875, 0.000274658203125, -0.00048828125, -0.000274658203125, -0.001007080078125, -0.00079345703125, -0.0008544921875, -0.001739501953125, 0.000701904296875, 0.001495361328125, -0.000885009765625, -0.0006103515625, 0.00048828125, -0.001678466796875, -0.001312255859375, ...], 'sampling_rate': 16000}",pro zodpovědnou volbu je také nutné mít dostatek informací.,96718,True
1,8,"{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f860858b1727f7dfcbcf1817dd95332e8fad098537c56802f61c4de2454c7def/train_part_0/20100119-0900-PLENARY-4-cs_20100119-15:41:43_5.wav', 'array': [-0.00250244140625, -0.003143310546875, -0.00079345703125, 0.0003662109375, -0.00152587890625, -0.001739501953125, -0.000885009765625, -0.001129150390625, -0.000457763671875, 0.00048828125, 0.0008544921875, 0.001800537109375, 0.0013427734375, 0.003082275390625, 0.0045166015625, 0.002593994140625, 0.002838134765625, 0.00244140625, 0.00140380859375, 0.000732421875, 0.000732421875, 6.103515625e-05, -0.000518798828125, -0.00030517578125, -0.000457763671875, 0.00091552734375, 0.0008544921875, -0.0003662109375, -0.00128173828125, 0.0001220703125, 0.000640869140625, -0.00201416015625, -0.001922607421875, 0.000762939453125, 0.0023193359375, 0.00128173828125, 0.00054931640625, 0.001922607421875, 0.002685546875, 0.0013427734375, 0.0, 0.00164794921875, 0.00201416015625, 0.000457763671875, 0.00115966796875, 0.00128173828125, -3.0517578125e-05, -0.0018310546875, -0.00201416015625, -0.001708984375, -0.002349853515625, -0.00140380859375, -0.0010986328125, -0.00054931640625, -0.00018310546875, -0.000885009765625, -0.00079345703125, -0.00128173828125, -0.000244140625, 0.00030517578125, -0.0001220703125, 3.0517578125e-05, -0.002197265625, -0.001953125, -0.000579833984375, -0.000885009765625, -9.1552734375e-05, 0.0, 0.000518798828125, -0.00048828125, -0.001251220703125, -0.000579833984375, -0.001617431640625, -0.002105712890625, -0.00274658203125, -0.00274658203125, -0.00146484375, 0.000213623046875, 0.00177001953125, 0.001251220703125, 0.00079345703125, 0.00225830078125, 0.002410888671875, 0.001861572265625, 0.001983642578125, 0.000396728515625, -0.00115966

In [ ]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["raw_text"] = re.sub(chars_to_remove_regex, '', batch["raw_text"]).lower()
    return batch

In [ ]:
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'speaker_id', 'is_gold_transcript'],
    num_rows: 3000
})

In [ ]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

In [ ]:
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'speaker_id', 'is_gold_transcript'],
    num_rows: 3000
})

In [ ]:
def replace_hatted_characters(batch):
  # train
  batch["normalized_text"] = re.sub('á', 'a', batch["normalized_text"])
  batch["normalized_text"] = re.sub('é', 'e', batch["normalized_text"])
  batch["normalized_text"] = re.sub('í', 'i', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ó', 'o', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ú', 'u', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ý', 'y', batch["normalized_text"])
  batch["normalized_text"] = re.sub('č', 'c', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ď', 'd', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ě', 'e', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ň', 'n', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ř', 'r', batch["normalized_text"])
  batch["normalized_text"] = re.sub('š', 's', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ť', 't', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ů', 'u', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ž', 'z', batch["normalized_text"])

  # test
  batch["normalized_text"] = re.sub('å', 'a', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ä', 'a', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ö', 'o', batch["normalized_text"])
  batch["normalized_text"] = re.sub('ć', 'c', batch["normalized_text"])
  batch["normalized_text"] = re.sub('č', 'c', batch["normalized_text"])
  batch["normalized_text"] = re.sub('đ', 'd', batch["normalized_text"])
  batch["normalized_text"] = re.sub('š', 's', batch["normalized_text"])
  # batch["normalized_text"] = re.sub('ž', 'z', batch["normalized_text"])

  return batch

In [ ]:
common_voice_train = common_voice_train.map(replace_hatted_characters)
common_voice_test = common_voice_test.map(replace_hatted_characters)

In [ ]:
# import requests
# from phonemizer import phonemize
# from phonemizer.separator import Separator

sentence = 'normalized_text'

# def ipa_encode(batch):
#   sentences = batch[sentence]
#   if type(sentences) is list:
#     sentences = "\n".join(sentences)
#   text = [line.strip() for line in sentences.split('\n') if line]

#   phn = phonemize(
#     text,
#     language='cs',
#     backend='espeak',
#     separator=Separator(phone=None, word=' ', syllable='|'),
#     strip=True,
#     preserve_punctuation=True)
#   batch[sentence] = phn
#   return batch

# # 整的需要ipa_encode吗？？？
# # using 'normalized_text' or raw? Could be able to use normalized_text directly without any processing, right? ? ?

In [ ]:
# common_voice_train = common_voice_train.map(ipa_encode, batched=True, batch_size=20, keep_in_memory=True)
# common_voice_test = common_voice_test.map(ipa_encode, batched=True, batch_size=20, keep_in_memory=True)

In [ ]:
show_random_elements(common_voice_train.remove_columns(["audio_id","audio"]))

,language,raw_text,normalized_text,speaker_id,is_gold_transcript
0,8,tím se tyto zbraně stanou potencionálně dostupné zločincům a teroristům,tim se tyto zbrane stanou potencionalne dostupne zlocincum a teroristum.,23704,True
1,8,pokud jde o arménii ta se rozhodla pro partnerství s ruskem a ázerbájdžán prakticky suspenduje práci v rámci východního partnerství,pokud jde o armenii ta se rozhodla pro partnerstvi s ruskem a azerbajdzan prakticky suspenduje praci v ramci vychodniho partnerstvi.,23706,True
2,8,a komise může mít stokrát pravdu pokud jde o formální dodržení procedur ale fatálně selhala pokud šlo řekněme o umělecký dojem nebo o image a poškodila svoji vlastní pověst vytvořila dojem že šlo o předem připravenou akci podle not pana junckera která má nějakým způsobem zachovat jeho dědictví až už nebude šéfem evropské komise,a komise muze mit stokrat pravdu pokud jde o formalni dodrzeni procedur ale fatalne selhala pokud slo rekneme o umelecky dojem nebo o image a poskodila svoji vlastni povest. vytvorila dojem ze slo o predem pripravenou akci podle not pana junckera ktera ma nejakym zpusobem zachovat jeho dedictvi az uz nebude sefem evropske komise.,23712,True
3,8,rámec pro provádění testu proporcionality bude tak vymezen jasně a zároveň nebude příliš komplikovaný po procesní stránce a nepřinese tak zbytečnou administrativní zátěž,ramec pro provadeni testu proporcionality bude tak vymezen jasne a zaroven nebude prilis komplikovany po procesni strance a neprinese tak zbytecnou administrativni zatez.,124710,True
4,8,musíme si uvědomit že i v letecké dopravě platí že nic není zadarmo,musime si uvedomit ze i v letecke doprave plati ze nic neni zadarmo.,28323,True
5,8,realitou je různorodost podmínek v členských zemích které mají své logické historické důvody ale jsou postaveny často i účelové zábrany chránící obchodní zájmy různých společností,realitou je ruznorodost podminek v clenskych zemich ktere maji sve logicke historicke duvody ale jsou postaveny casto i ucelove zabrany chranici obchodni zajmy ruznych spolecnosti.,96713,True
6,8,poslední poznámka vidím že zde nemáme zástupce členských států zástupce rady považuji to za velkou chybu,posledni poznamka vidim ze zde nemame zastupce clenskych statu zastupce rady povazuji to za velkou chybu.,124708,True
7,8,jako místopředsedkyně výboru pro kulturu a vzdělávání je pro mě podstatné že touto dohodou nebude dotčena možnost evropské unie poskytovat dotace a finanční podporu kulturním odvětvím,jako mistopredsedkyne vyboru pro kulturu a vzdelavani je pro me podstatne ze touto dohodou nebude dotcena moznost evropske unie poskytovat dotace a financni podporu kulturnim odvetvim.,124710,True
8,8,jedná se vlastně o neomezené bezcelní přístupy které jsou jednostranné a autonomní,jedna se vlastne o neomezene bezcelni pristupy ktere jsou jednostranne a autonomni.,124704,True
9,8,není důvod oddalovat zveřejnění nedůvěrné verze žádosti u které ani samotný žadatel nepředpokládá že by ho zveřejnění nějak poškodilo,neni duvod oddalovat zverejneni neduverne verze zadosti u ktere ani samotny zadatel nepredpoklada ze by ho zverejneni nejak poskodilo.,96715,True


In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["normalized_text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/478 [00:00<?, ? examples/s]

In [ ]:
common_voice_train[0]

{'audio_id': '20140204-0900-PLENARY-23-cs_20140204-22:43:32_1',
 'language': 8,
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/f860858b1727f7dfcbcf1817dd95332e8fad098537c56802f61c4de2454c7def/train_part_0/20140204-0900-PLENARY-23-cs_20140204-22:43:32_1.wav',
  'array': array([-0.00384521, -0.00048828,  0.00222778, ...,  0.00286865,
          0.00192261,  0.00024414]),
  'sampling_rate': 16000},
 'raw_text': 'musí být transparentní a srozumitelný firmám musí zaručovat právní jistotu musí usilovat o odstraňování zbytečných byrokratických překážek ale současně musí umět zaručit a vymoci i vysokou úroveň ochrany spotřebitelů a zejména jejich bezpečnost',
 'normalized_text': 'musi byt transparentni a srozumitelny firmam musi zarucovat pravni jistotu musi usilovat o odstranovani zbytecnych byrokratickych prekazek ale soucasne musi umet zarucit a vymoci i vysokou uroven ochrany spotrebitelu a zejmena jejich bezpecnost.',
 'speaker_id': '96718',
 'is_gold_transcript'

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 '!': 1,
 "'": 2,
 '.': 3,
 '?': 4,
 'a': 5,
 'b': 6,
 'c': 7,
 'd': 8,
 'e': 9,
 'f': 10,
 'g': 11,
 'h': 12,
 'i': 13,
 'j': 14,
 'k': 15,
 'l': 16,
 'm': 17,
 'n': 18,
 'o': 19,
 'p': 20,
 'q': 21,
 'r': 22,
 's': 23,
 't': 24,
 'u': 25,
 'v': 26,
 'w': 27,
 'x': 28,
 'y': 29,
 'z': 30}

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

33

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
repo_name = "cs2fi_wav2vec2-large-xls-r-300m-czech-colab"

In [ ]:
tokenizer.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/Roxysun/cs2fi_wav2vec2-large-xls-r-300m-czech-colab/commit/22cf056341db238e3c492035d2478c229db90ff8', commit_message='Upload tokenizer', commit_description='', oid='22cf056341db238e3c492035d2478c229db90ff8', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
common_voice_train

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'speaker_id', 'is_gold_transcript'],
    num_rows: 3000
})

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f860858b1727f7dfcbcf1817dd95332e8fad098537c56802f61c4de2454c7def/train_part_0/20140204-0900-PLENARY-23-cs_20140204-22:43:32_1.wav',
 'array': array([-0.00384521, -0.00048828,  0.00222778, ...,  0.00286865,
         0.00192261,  0.00024414]),
 'sampling_rate': 16000}

In [ ]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [ ]:
common_voice_train[0]["audio"]

{'path': '/root/.cache/huggingface/datasets/downloads/extracted/f860858b1727f7dfcbcf1817dd95332e8fad098537c56802f61c4de2454c7def/train_part_0/20140204-0900-PLENARY-23-cs_20140204-22:43:32_1.wav',
 'array': array([-0.00384521, -0.00048828,  0.00222778, ...,  0.00286865,
         0.00192261,  0.00024414]),
 'sampling_rate': 16000}

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int][sentence])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

dezinformacemi byly zaplaveny a ovlivneny i prezidentske volby v me zemi ceske republice.


In [ ]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int][sentence])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: z hlediska srovnani s dunajskou strategii neni mozne zahrnout karpatskou strategii k dunaji protoze se jedna uplne o jine podminky.
Input array shape: (143039,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch[sentence]).input_ids

    return batch

In [ ]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

In [ ]:
common_voice_train

Dataset({
    features: ['input_values', 'input_length', 'labels'],
    num_rows: 3000
})

In [ ]:
# max_input_length_in_sec = 5.0
max_input_length_in_sec = 5.0
common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [ ]:
max_input_length_in_sec = 5.0
common_voice_test = common_voice_test.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

<ipython-input-48-605de035dfd7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

Some weights of the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-lv-60-espeak-cv-ft and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably

In [ ]:
# from transformers import Wav2Vec2ForCTC

# model = Wav2Vec2ForCTC.from_pretrained(
#     "facebook/wav2vec2-lv-60-espeak-cv-ft",
#     attention_dropout=0.0,
#     hidden_dropout=0.0,
#     feat_proj_dropout=0.0,
#     mask_time_prob=0.05,
#     layerdrop=0.0,
#     ctc_loss_reduction="mean",
#     pad_token_id=processor.tokenizer.pad_token_id,
#     vocab_size=len(processor.tokenizer),
# )

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1913: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5. Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments

# Not sure. using the name after finetune training here? ? ?

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50, #50
  gradient_checkpointing=True,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=100,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


{'eval_loss': 7530.18212890625,
 'eval_wer': 1.0020964360587001,
 'eval_runtime': 4.453,
 'eval_samples_per_second': 14.372,
 'eval_steps_per_second': 1.797}

In [ ]:
torch.cuda.empty_cache()
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
100,3042.873800,422.193817,0.951782
200,362.155400,231.748596,1.000000
300,208.092000,196.419373,0.995807
400,189.135400,211.622314,0.935010
500,163.635500,235.320129,0.918239
600,140.795900,256.402802,0.953878
700,115.550600,311.456207,1.014675
800,93.662900,304.088196,1.224319
900,78.969400,354.541504,1.127883
1000,67.415100,423.617798,1.085954


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Tran

Step,Training Loss,Validation Loss,Wer
100,3042.873800,422.193817,0.951782
200,362.155400,231.748596,1.000000
300,208.092000,196.419373,0.995807
400,189.135400,211.622314,0.935010
500,163.635500,235.320129,0.918239
600,140.795900,256.402802,0.953878
700,115.550600,311.456207,1.014675
800,93.662900,304.088196,1.224319
900,78.969400,354.541504,1.127883
1000,67.415100,423.617798,1.085954


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=1400, training_loss=333.1342972237723, metrics={'train_runtime': 2627.088, 'train_samples_per_second': 8.641, 'train_steps_per_second': 0.533, 'total_flos': 2.46684733278004e+18, 'train_loss': 333.1342972237723, 'epoch': 49.12})

In [ ]:
# upload the model after finetune
trainer.push_to_hub(repo_name)

'https://huggingface.co/Roxysun/cs2fi_wav2vec2-large-xls-r-300m-czech-colab/tree/main/'